In [26]:
import pandas as pd
import numpy as np
from ModelingUtils import make_dummies, make_ml_target
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.preprocessing import label_binarize

In [27]:
data = pd.read_parquet('../data/prepared_data.parquet')

### make ml target

In [28]:
data = make_ml_target(data)
data = make_dummies(data, 'near_city_station_name')

In [29]:
data

,pk,Relacja,Stacja,Opóźnienie przyjazdu,Opóźnienie odjazdu,arrival_on_time,departure_on_time,lat,lon,station_count_on_curr_station,...,near_city_station_name_Katowice,near_city_station_name_Kołobrzeg,near_city_station_name_Lublin Główny,near_city_station_name_Olsztyn Główny,near_city_station_name_Poznań Główny,near_city_station_name_Rzeszów Główny,near_city_station_name_Szczecin Główny,near_city_station_name_Warszawa Centralna,near_city_station_name_Wrocław Główny,near_city_station_name_Łódź Widzew
0,1,Przeworsk - Rzeszów Główny,Przeworsk,0,0,2022-12-01 06:55:00,2022-12-01 06:55:00,50.067161,22.503232,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,Przeworsk - Rzeszów Główny,Grzęska,0,0,2022-12-01 06:58:30,2022-12-01 06:59:00,50.080037,22.452564,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,Przeworsk - Rzeszów Główny,Rogóżno koło Łańcuta,0,0,2022-12-01 07:03:00,2022-12-01 07:03:30,50.083573,22.362513,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1,Przeworsk - Rzeszów Główny,Kosina,0,0,2022-12-01 07:06:30,2022-12-01 07:07:00,50.083570,22.322846,3,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1,Przeworsk - Rzeszów Główny,Głuchów,0,0,2022-12-01 07:09:30,2022-12-01 07:10:00,50.083469,22.279077,4,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138205,52128,Kraków Lotnisko - Wieliczka Rynek-Kopalnia,Kraków Bieżanów,0,0,2023-04-23 08:52:12,2023-04-23 08:52:42,50.021694,20.029257,10,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1138206,52128,Kraków Lotnisko - Wieliczka Rynek-Kopalnia,Kraków Bieżanów Drożdżownia,0,0,2023-04-23 08:55:00,2023-04-23 08:55:30,50.010696,20.035373,11,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1138207,52128,Kraków Lotnisko - Wieliczka Rynek-Kopalnia,Wieliczka Bogucice,0,0,2023-04-23 08:57:24,2023-04-23 08:57:54,49.998474,20.036696,12,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1138208,52128,Kraków Lotnisko - Wieliczka Rynek-Kopalnia,Wieliczka Park,0,0,2023-04-23 09:00:06,2023-04-23 09:00:36,49.989105,20.049334,13,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
y = data['ML_TARGET'].values
# y = label_binarize(y, classes=[0,1,2,3])
# n_classes = y.shape[1]

X = data.iloc[:,9:].drop('ML_TARGET', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
model = CatBoostClassifier(
    iterations=1000,  # Number of trees (boosting rounds)
    learning_rate=0.1,  # Learning rate
    depth=6,  # Depth of each tree
    loss_function='MultiClass',  # Loss function for multiclass classification
    random_seed=123
)
model.fit(X_train, y_train)

0:	learn: 1.1960653	total: 1.1s	remaining: 18m 22s
1:	learn: 1.0620177	total: 1.58s	remaining: 13m 9s
2:	learn: 0.9617336	total: 1.86s	remaining: 10m 18s
3:	learn: 0.8840828	total: 2.14s	remaining: 8m 53s
4:	learn: 0.8218968	total: 2.42s	remaining: 8m 2s
5:	learn: 0.7706168	total: 2.71s	remaining: 7m 29s
6:	learn: 0.7273040	total: 3.06s	remaining: 7m 13s
7:	learn: 0.6907161	total: 3.53s	remaining: 7m 17s
8:	learn: 0.6601644	total: 3.83s	remaining: 7m 1s
9:	learn: 0.6339126	total: 4.12s	remaining: 6m 48s
10:	learn: 0.6109925	total: 4.44s	remaining: 6m 38s
11:	learn: 0.5913708	total: 4.74s	remaining: 6m 30s
12:	learn: 0.5744138	total: 5.08s	remaining: 6m 26s
13:	learn: 0.5600061	total: 5.41s	remaining: 6m 21s
14:	learn: 0.5472621	total: 5.74s	remaining: 6m 16s
15:	learn: 0.5359101	total: 6.09s	remaining: 6m 14s
16:	learn: 0.5258017	total: 6.42s	remaining: 6m 11s
17:	learn: 0.5173056	total: 6.79s	remaining: 6m 10s
18:	learn: 0.5097049	total: 7.07s	remaining: 6m 4s
19:	learn: 0.5026930	tot

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
model = RandomForestClassifier(random_state=123)

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

cv = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
cv.fit(X_train, y_train)